In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms, models
from torchvision.transforms import autoaugment
from torch.utils.data import DataLoader, random_split
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau, StepLR
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
import itertools

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

Using cuda device


In [5]:
transform = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
}

In [6]:
data_dir = "/content/drive/MyDrive/dressmeup/cloths"

In [7]:
image_datasets = {x: datasets.ImageFolder(root=data_dir+'/'+x, transform=transform[x])
                  for x in ['train', 'valid', 'test']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=2)
               for x in ['train', 'valid', 'test']}

In [8]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 6)


# model.fc = nn.Sequential(
#     nn.Linear(model.fc.in_features, 512),
#     nn.ReLU(),
#     nn.Dropout(0.4),
#     nn.Linear(512, 2)
# )


model = model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 157MB/s]


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
step_lr_scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

In [10]:
def train(model, criterion, optimizer, scheduler, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        # Training phase
        for inputs, labels in dataloaders['train']:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        # Update the learning rate scheduler after each epoch
        scheduler.step()

        # Calculate average training loss for the epoch
        epoch_loss = running_loss / len(image_datasets['train'])
        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}')

        # Validation phase
        model.eval()
        corrects = 0
        for inputs, labels in dataloaders['valid']:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            corrects += torch.sum(preds == labels.data)

        # Calculate validation accuracy for the epoch
        epoch_acc = corrects.double() / len(image_datasets['valid'])
        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Acc: {epoch_acc:.4f}')

In [11]:
def test(model, dataloader):
    model.eval()
    corrects = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader['test']:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            corrects += torch.sum(preds == labels.data)
            total += labels.size(0)

    test_acc = corrects.double() / total
    print(f'Test Accuracy: {test_acc:.4f}')

In [12]:
# 학습 및 스케줄러 갱신 수행
train(model, criterion, optimizer, step_lr_scheduler, num_epochs=25)

# 테스트 데이터에 대한 정확도 출력
test(model, dataloaders)

Epoch [1/25], Training Loss: 1.2859
Epoch [1/25], Validation Acc: 0.8394
Epoch [2/25], Training Loss: 0.8241
Epoch [2/25], Validation Acc: 0.8759
Epoch [3/25], Training Loss: 0.6919
Epoch [3/25], Validation Acc: 0.8613
Epoch [4/25], Training Loss: 0.6205
Epoch [4/25], Validation Acc: 0.8686
Epoch [5/25], Training Loss: 0.5628
Epoch [5/25], Validation Acc: 0.8905
Epoch [6/25], Training Loss: 0.5623
Epoch [6/25], Validation Acc: 0.9075
Epoch [7/25], Training Loss: 0.5026
Epoch [7/25], Validation Acc: 0.9027
Epoch [8/25], Training Loss: 0.4558
Epoch [8/25], Validation Acc: 0.9319
Epoch [9/25], Validation Acc: 0.9270
Epoch [10/25], Training Loss: 0.4306
Epoch [10/25], Validation Acc: 0.9221
Epoch [11/25], Training Loss: 0.4141
Epoch [11/25], Validation Acc: 0.9221
Epoch [12/25], Training Loss: 0.3956
Epoch [12/25], Validation Acc: 0.9221
Epoch [13/25], Training Loss: 0.4159
Epoch [13/25], Validation Acc: 0.9270
Epoch [14/25], Training Loss: 0.4329
Epoch [14/25], Validation Acc: 0.9294
Epoc

In [15]:
torch.save(model.state_dict(), '/content/drive/MyDrive/dressmeup/cloth-segmentation/long_short_classification.pth')

# 모델 로드
loaded_model = models.resnet18(pretrained=False)
loaded_model.fc =  nn.Linear(model.fc.in_features, 6).to(device)

loaded_model.load_state_dict(torch.load('/content/drive/MyDrive/dressmeup/cloth-segmentation/long_short_classification.pth'))
loaded_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
from PIL import Image
import torchvision.transforms as transforms
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


image_path = "/content/drive/MyDrive/dressmeup/cloth-segmentation/output/2988164_1_220.jpg"
image = Image.open(image_path)


preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

input_tensor = preprocess(image)
input_batch = input_tensor.unsqueeze(0)  # 모델은 배치를 사용하므로 차원 추가

# 저장한 모델 불러오기
loaded_model = models.resnet18(pretrained=False)
loaded_model.fc = nn.Linear(loaded_model.fc.in_features, 6)
loaded_model = loaded_model.to(device)

loaded_model.load_state_dict(torch.load('/content/drive/MyDrive/dressmeup/cloth-segmentation/long_short_classification.pth', map_location=device))
loaded_model.eval()


input_batch = input_batch.to(device)


with torch.no_grad():
    output = loaded_model(input_batch)


_, predicted_class = torch.max(output, 1)
predicted_class_index = predicted_class.item()


class_labels = image_datasets['train'].class_to_idx


predicted_class_name = next(key for key, value in class_labels.items() if value == predicted_class_index)
print(f"Predicted Class Name: {predicted_class_name}")